In [ ]:
import tkinter as tk
import threading
import time
from PIL import Image, ImageTk
import numpy as np
import os
import sys

root = tk.Tk()


def center_window(root, width, height):
    screen_width = root.winfo_screenwidth()
    screen_height = root.winfo_screenheight()
    x = (screen_width - width) // 2
    y = (screen_height - height) // 2
    root.geometry(f'{width}x{height}+{x}+{y}')

root.title("8toN_Queen")
center_window(root, 960, 640)
root.configure(bg='#161512')
root.resizable(False, False)

BOARD_SIZE = 8
BOARD_SIDE = 480
BOARD_x = 50
BOARD_y = 50
chess_board = []
img_list = []



class Point_value:
    def _init_(self, x, y):
        self.x = x
        self.y = y

Showing_board = []
solve_sum = 0
#Running_board = []
running = True



def is_number(s):
    try:
        int(s)
        return True
    except ValueError:
        return False

def size_return():
    value = size_set.get()
    if not is_number(value):
        signal_window = tk.Toplevel(root)
        signal_window.title("错误")
        center_window(signal_window, 160, 50)
        signal_window.resizable(False, False)
        label = tk.Label(signal_window, text="请输入有效的数字", 
                 bg='#161512',  fg='#cacaca',  font=("黑体", 12, 'bold'), 
                 width=20,  height=5)
        signal_window.grid_rowconfigure(0, weight=1)
        signal_window.grid_columnconfigure(0, weight=1)
        label.grid(row=0, column=0, sticky="nsew")
    else:
        size = int(value)
        if size > 26 or size < 1:
            signal_window = tk.Toplevel(root)
            signal_window.title("错误")
            center_window(signal_window, 160, 50)
            signal_window.resizable(False, False)
            label = tk.Label(signal_window, text="数字超出范围", 
                     bg='#161512',  fg='#cacaca',  font=("黑体", 12, 'bold'), 
                     width=20,  height=5)
            signal_window.grid_rowconfigure(0, weight=1)
            signal_window.grid_columnconfigure(0, weight=1)
            label.grid(row=0, column=0, sticky="nsew")
        else:
            global BOARD_SIZE
            BOARD_SIZE = size
            global Q_img
            global Q_img_show
            Q_img = Q_img.resize((BOARD_SIDE // BOARD_SIZE - 4, BOARD_SIDE // BOARD_SIZE - 4), Image.LANCZOS)
            Q_img_show = ImageTk.PhotoImage(Q_img)
            create_chessboard(root)

def resource_path(relative_path):
    try:
        base_path = sys._MEIPASS
    except Exception:
        base_path = os.path.abspath(".")
    return os.path.join(base_path, relative_path)
img_path = resource_path("LightQueen.png")
Q_img = Image.open(img_path)
Q_img = Q_img.convert("RGBA")
Q_img = Q_img.resize((BOARD_SIDE // BOARD_SIZE - 4, BOARD_SIDE // BOARD_SIZE - 4), Image.LANCZOS)
Q_img_show = ImageTk.PhotoImage(Q_img)

    

def is_safe(board, row, col):
    for i in range(col):
        if board[row][i] == 1:
            return False
    for i, j in zip(range(row, -1, -1), range(col, -1, -1)):
        if board[i][j] == 1:
            return False
    for i, j in zip(range(row, BOARD_SIZE, 1), range(col, -1, -1)):
        if board[i][j] == 1:
            return False
    return True



def solve_queen(board, col):
    global solve_sum
    if not running:
        return
    if col >= BOARD_SIZE:
        string_result = []
        for i in range(BOARD_SIZE):
            for j in range(BOARD_SIZE):
                if board[i][j] == 1:
                    if BOARD_SIZE < 10:
                        string_result.append(chr(j + 65))
                        string_result.append(chr(BOARD_SIZE - i + 48))
                        string_result.append(' ')
                    else:
                        string_result.append(chr(j + 65))
                        string_result.append(chr((BOARD_SIZE - i) // 10 + 48))
                        string_result.append(chr((BOARD_SIZE - i) % 10 + 48))
                        string_result.append(' ')
        string_get = ''.join(string_result)
        solve_sum = solve_sum + 1
        list_solution.insert(tk.END, string_get)
        return
    for i in range(BOARD_SIZE):
        if is_safe(board, i, col):
            board[i][col] = 1
            color = '#f0d9b5' if (i + col) % 2 == 0 else '#b58863'
            label = tk.Label(root, image=Q_img_show, bg = color)
            label.place(x=Showing_board[i][col].x, y=Showing_board[i][col].y)
            root.update_idletasks()
            solve_queen(board, col + 1)
            board[i][col] = 0
            label.destroy()
            label = tk.Canvas(root, width=BOARD_SIDE // BOARD_SIZE, height=BOARD_SIDE // BOARD_SIZE, bg=color, highlightthickness=0)
            label.place(x=Showing_board[i][col].x, y=Showing_board[i][col].y)
            root.update_idletasks()
            label.destroy()

def print_result(result_string):
    border_canvas = tk.Canvas(root, width=(BOARD_SIDE + 8), height=(BOARD_SIDE + 8))
    border_canvas.place(x=BOARD_x - 6, y=BOARD_y - 6)
    border_color = "#2a2724"
    border_thickness = 20
    border_canvas.create_rectangle(0, 0, BOARD_SIDE, BOARD_SIDE,
                                   outline=border_color, width=border_thickness)
    square_size = BOARD_SIDE // BOARD_SIZE
    for row in range(BOARD_SIZE):
        for col in range(BOARD_SIZE):
            color = '#f0d9b5' if (row + col) % 2 == 0 else '#b58863'
            square = tk.Canvas(root, width=square_size, height=square_size, bg=color, highlightthickness=0)
            square.place(x=BOARD_x + col * square_size, y=BOARD_y + row * square_size)
            
    print_point = [0 for _ in range(BOARD_SIZE)]
    if BOARD_SIZE < 10:
        for index in range(0, len(result_string), 3):
            point_place = result_string[index:index+3]
            print_point[index // 3] = [BOARD_SIZE - (ord(point_place[0]) - 65), int(point_place[1]) - 1]
    else:
        for index in range(0, len(result_string), 4):
            point_place = result_string[index:index+4]
            print_point[index // 4] = [BOARD_SIZE - (ord(point_place[0]) - 65), 
                                       int(point_place[1]) * 10 + int(point_place[2]) - 1]
    def get_color(point):
        color = '#f0d9b5' if (point[0] + point[1]) % 2 == 0 else '#b58863'
        return color

    print(print_point)

    for select_point in range(BOARD_SIZE):
        label = tk.Label(root, image=Q_img_show, bg = get_color(print_point[select_point]))
        label.place(x=Showing_board[print_point[select_point][1]][print_point[select_point][0]].x, 
                    y=Showing_board[print_point[select_point][1]][print_point[select_point][0]].y)
    root.update_idletasks()
        




def finished_signal():
    signal_window = tk.Toplevel(root)
    signal_window.title("已完成" if running else "已中断")
    center_window(signal_window, 160, 50)
    signal_window.resizable(False, False)
    label = tk.Label(signal_window, text="已结束：共" + str(solve_sum) + "个解", 
             bg='#161512',  fg='#cacaca',  font=("黑体", 12, 'bold'), 
             width=20,  height=5)
    signal_window.grid_rowconfigure(0, weight=1)
    signal_window.grid_columnconfigure(0, weight=1)
    label.grid(row=0, column=0, sticky="nsew")

def solving():
    solve_queen(chess_board, 0)
    buttons[2].config(state=tk.NORMAL)
    finished_signal()


def run_order():
    size_set.delete(0, tk.END)
    size_set.insert(0, str(BOARD_SIZE))
    buttons[0].config(state=tk.DISABLED)
    buttons[1].config(state=tk.DISABLED)
    solving()

def reset_order():
    chess_board = [[0 for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
    global solve_sum
    solve_sum = 0
    buttons[0].config(state=tk.NORMAL)
    buttons[1].config(state=tk.NORMAL)
    buttons[2].config(state=tk.DISABLED)
    list_solution.delete(0, tk.END)

    border_canvas = tk.Canvas(root, width=(BOARD_SIDE + 8), height=(BOARD_SIDE + 8))
    border_canvas.place(x=BOARD_x - 6, y=BOARD_y - 6)
    border_color = "#2a2724"
    border_thickness = 20
    border_canvas.create_rectangle(0, 0, BOARD_SIDE, BOARD_SIDE,
                                   outline=border_color, width=border_thickness)
    square_size = BOARD_SIDE // BOARD_SIZE
    for row in range(BOARD_SIZE):
        row_point = []
        for col in range(BOARD_SIZE):
            color = '#f0d9b5' if (row + col) % 2 == 0 else '#b58863'
            square = tk.Canvas(root, width=square_size, height=square_size, bg=color, highlightthickness=0)
            square.place(x=BOARD_x + col * square_size, y=BOARD_y + row * square_size)


def show_result(evt):
    w = evt.widget
    indices = w.curselection()
    if indices:
        for index in indices:
            value = w.get(index)
            print_result(value)
            

def create_chessboard(parent):
    back_label = tk.Label(parent, width=70, height=(BOARD_SIDE + 5), bg = "#161512", fg = "#161512")
    back_label.place(x=BOARD_x - 35, y=BOARD_y - 8)
    global chess_board
    global Showing_board
    Showing_board = []
    chess_board = [[0 for _ in range(BOARD_SIZE)] for _ in range(BOARD_SIZE)]
    border_canvas = tk.Canvas(parent, width=(BOARD_SIDE + 8), height=(BOARD_SIDE + 8))
    border_canvas.place(x=BOARD_x - 6, y=BOARD_y - 6)
    border_color = "#2a2724"
    border_thickness = 20
    border_canvas.create_rectangle(0, 0, BOARD_SIDE, BOARD_SIDE,
                                   outline=border_color, width=border_thickness)
    square_size = BOARD_SIDE // BOARD_SIZE
    for row in range(BOARD_SIZE):
        row_point = []
        for col in range(BOARD_SIZE):
            color = '#f0d9b5' if (row + col) % 2 == 0 else '#b58863'
            square = tk.Canvas(parent, width=square_size, height=square_size, bg=color, highlightthickness=0)
            square.place(x=BOARD_x + col * square_size, y=BOARD_y + row * square_size)
            row_point.append(Point_value())
            row_point[col].x = BOARD_x + col * square_size
            row_point[col].y = BOARD_y + row * square_size
        Showing_board.append(row_point)
    for row in range(BOARD_SIZE):
        label = tk.Label(parent, text = str(BOARD_SIZE - row))
        label.configure(bg="#161512", fg="#FFFFFF", font=("Arial", 12, 'bold'))
        label.place(x = BOARD_x - 35, y = BOARD_y + square_size // 3 + row * square_size)
    for col in range(BOARD_SIZE):
        label = tk.Label(parent, text = chr(col + 65))
        label.configure(bg="#161512", fg="#FFFFFF", font=("Arial", 12, 'bold'))
        label.place(x = BOARD_x + square_size // 3 + col * square_size, y = BOARD_y + BOARD_SIDE + 15)
        


create_chessboard(root)
#create_chessboard(root, Running_board)

size_set = tk.Entry(root, bg="#e1cfc1", fg="#302b24", font=("Arial", 12, 'bold'), width=8)
size_set.insert(0, str(BOARD_SIZE))
size_set.place(x=740, y=60)
size_set.bind('<Return>', lambda event: size_return())
size_label = tk.Label(root, bg="#161512", fg="#FFFFFF", font=("黑体", 12, 'bold'), text = "尺寸：")
size_label.place(x=680, y=60)

buttons = [tk.Button(root, text="确认", command=size_return), 
           tk.Button(root, text="求解", command=run_order),  
           tk.Button(root, text="重置", command=reset_order, state=tk.DISABLED)]
for button in buttons:
    button.configure(compound='left', bg="#51504d", fg="#cacaca", font=("黑体", 12, 'bold'), height=1, width=8)
buttons[0].place(x=740, y=100)
buttons[1].place(x=660, y=160)
buttons[2].place(x=760, y=160)

list_solution = tk.Listbox(root)
list_solution.configure(bg="#e1cfc1", fg="#302b24", font=("Arial", 12, 'bold'), height=6, width=25)
list_solution.place(x=635, y=250)
list_solution.bind('<<ListboxSelect>>', show_result)
solutions_label = tk.Label(root, bg="#161512", fg="#FFFFFF", font=("黑体", 12, 'bold'), text = "解列表：")
solutions_label.place(x=635, y=220)

root.mainloop()

[[3, 5], [6, 4], [2, 3], [5, 2], [1, 1], [4, 0]]


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\lenovo\.conda\envs\linkstart\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\lenovo\AppData\Local\Temp\ipykernel_2304\1366249631.py", line 247, in show_result
    print_result(value)
  File "C:\Users\lenovo\AppData\Local\Temp\ipykernel_2304\1366249631.py", line 184, in print_result
    label.place(x=Showing_board[print_point[select_point][1]][print_point[select_point][0]].x,
IndexError: list index out of range
